In [2]:
import numpy as np
import pandas as pd
import torch as torch
from torch import nn

import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
data_path = '/content/sample_data/data'

In [3]:

import os

train_data = datasets.FashionMNIST(
     root = data_path,
     train = True,
     download = True,
     transform = ToTensor()
     )
test_data = datasets.FashionMNIST(
     root = data_path,
     train = False,
     download = True,
     transform = ToTensor()
 )

100%|██████████| 26421880/26421880 [00:01<00:00, 16607146.28it/s]


Extracting /content/sample_data/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/sample_data/data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 269223.49it/s]


Extracting /content/sample_data/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/sample_data/data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5081386.53it/s]


Extracting /content/sample_data/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/sample_data/data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3783472.40it/s]

Extracting /content/sample_data/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/sample_data/data/FashionMNIST/raw



In [ ]:
import os

# Check if the dataset files are in the correct location

expected_files = [
    'train-images-idx3-ubyte',
    'train-labels-idx1-ubyte',
    't10k-images-idx3-ubyte',
    't10k-labels-idx1-ubyte'
]

for filename in expected_files:
    file_path = os.path.join(data_path, filename)
    if os.path.exists(file_path):
        print(f"{file_path} exists.")
    else:
        print(f"{file_path} does not exist.")


In [4]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [5]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'

)
print(f"Using {device} device")

Using cpu device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.Relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)

        )
    def forward(self, x):
       x = self.flatten(x)
       logits = self.Relu_stack(x)
       return logits
model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:

model = NeuralNetwork().to(device)

# Now the 'model' variable is defined and can be used
loss_fn = nn.CrossEntropyLoss()
optimzer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(DataLoader, model, loss_fn, optimizer):
  size = len(DataLoader.dataset)
  model.train()
  for batch, (X,y) in enumerate(DataLoader):
    X, y = X.to(device), y.to(device)

    Pred = model(X)
    loss = loss_fn(Pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")



In [10]:
def test(DataLoader, model, loss_fn):
  size = len(DataLoader.dataset)
  num_batches = len(DataLoader)
  model.eval()
  test_loss = 0  # Initialize test_loss to 0
  correct = 0
  with torch.no_grad():
    for X, y in DataLoader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss  += loss_fn( pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
      test_loss /= num_batches
      correct /= size
      print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
Epochs = 5
for t in range(Epochs):
  print(f'Epoch {t+1}\n-------------------------------')
  train(train_dataloader,model,loss_fn,optimzer)
  test(test_dataloader,model,loss_fn)

Epoch 1
-------------------------------
loss: 2.300508 [   64/60000]
loss: 2.285744 [ 6464/60000]
loss: 2.279085 [12864/60000]
loss: 2.254410 [19264/60000]
loss: 2.246394 [25664/60000]
loss: 2.227254 [32064/60000]
loss: 2.227679 [38464/60000]
loss: 2.180512 [44864/60000]
loss: 2.171160 [51264/60000]
loss: 2.145336 [57664/60000]
Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013763 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013833 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013872 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013897 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013745 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013894 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013913 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013914 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013890 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013819 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013895 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013658 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013902 

Test Er

In [12]:
torch.save(model.state_dict(),'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')